In [1]:
# Primary libraries
from time import time
import numpy as np
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

# Measures
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import f1_score, accuracy_score, recall_score, confusion_matrix, precision_score, ConfusionMatrixDisplay, classification_report
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv("./dataset/final_data.csv", index_col=0)
df.tail()

,match_api_id,home_team_buildUpPlaySpeed,home_team_buildUpPlayPassing,home_team_chanceCreationPassing,home_team_chanceCreationCrossing,home_team_chanceCreationShooting,home_team_defencePressure,home_team_defenceAggression,home_team_defenceTeamWidth,home_team_avg_shots,...,away_player_3_overall_rating,away_player_4_overall_rating,away_player_5_overall_rating,away_player_6_overall_rating,away_player_7_overall_rating,away_player_8_overall_rating,away_player_9_overall_rating,away_player_10_overall_rating,away_player_11_overall_rating,outcome
13209,2030167.0,57.0,50.0,61.0,62.0,49.0,49.0,42.0,52.0,11.158163,...,82.0,80.0,80.0,82.0,79.0,77.0,80.0,79.0,75.0,Win
13210,2030168.0,42.0,36.0,41.0,40.0,50.0,47.0,37.0,52.0,9.678322,...,75.0,77.0,75.0,75.0,77.0,76.0,74.0,73.0,78.0,Win
13211,2030169.0,46.0,39.0,50.0,60.0,36.0,49.0,57.0,50.0,10.117647,...,74.0,76.0,73.0,71.0,75.0,73.0,76.0,77.0,72.0,Win
13212,2030170.0,57.0,59.0,61.0,63.0,51.0,48.0,45.0,62.0,9.684211,...,72.0,76.0,71.0,79.0,75.0,71.0,75.0,72.0,79.0,Draw
13213,2030171.0,29.0,44.0,44.0,44.0,47.0,54.0,52.0,63.0,11.293478,...,75.0,76.0,71.0,76.0,77.0,74.0,75.0,73.0,78.0,Win


In [3]:
matches = pd.read_csv("./dataset/matches.csv", index_col=0)
matches = matches.drop(['id'], axis = 1)
matches = matches[['country_id', 'league_id', 'season', 'stage', 'date', 'match_api_id', 'home_team_api_id', 'away_team_api_id']]
matches.tail()

,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id
25974,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992091,10190,10191
25975,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992092,9824,10199
25976,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992093,9956,10179
25977,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992094,7896,10243
25978,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992095,10192,9931


In [4]:
with sqlite3.connect("dataset/database.sqlite") as con:
    team = pd.read_sql_query("SELECT distinct * from team",con)

team.tail()

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
294,49479,10190,898.0,FC St. Gallen,GAL
295,49837,10191,1715.0,FC Thun,THU
296,50201,9777,324.0,Servette FC,SER
297,50204,7730,1862.0,FC Lausanne-Sports,LAU
298,51606,7896,NaN,Lugano,LUG


In [5]:
team = team.drop(['id'], axis=1)

In [6]:
matches = pd.merge(matches, team, left_on='home_team_api_id', right_on='team_api_id')
matches = matches.drop(['team_api_id', 'team_fifa_api_id'], axis=1).rename(columns = {'team_long_name': 'home_team_long_name', 'team_short_name': 'home_team_short_name'})
matches.tail()

,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_long_name,home_team_short_name
25974,24558,24558,2015/2016,34,2016-05-16 00:00:00,1992217,7896,10192,Lugano,LUG
25975,24558,24558,2015/2016,36,2016-05-25 00:00:00,1992226,7896,10190,Lugano,LUG
25976,24558,24558,2015/2016,6,2015-08-22 00:00:00,1992076,7896,9931,Lugano,LUG
25977,24558,24558,2015/2016,7,2015-08-29 00:00:00,1992081,7896,10199,Lugano,LUG
25978,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992094,7896,10243,Lugano,LUG


In [7]:
matches = pd.merge(matches, team, left_on='away_team_api_id', right_on='team_api_id')
matches = matches.drop(['team_api_id', 'team_fifa_api_id'], axis=1).rename(columns = {'team_long_name': 'away_team_long_name', 'team_short_name': 'away_team_short_name'})
matches.tail()

,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_long_name,home_team_short_name,away_team_long_name,away_team_short_name
25974,24558,24558,2012/2013,8,2012-09-01 00:00:00,1227772,7730,9956,FC Lausanne-Sports,LAU,Grasshopper Club Zürich,GRA
25975,24558,24558,2013/2014,29,2014-04-12 00:00:00,1471923,7730,9956,FC Lausanne-Sports,LAU,Grasshopper Club Zürich,GRA
25976,24558,24558,2013/2014,4,2013-08-03 00:00:00,1471649,7730,9956,FC Lausanne-Sports,LAU,Grasshopper Club Zürich,GRA
25977,24558,24558,2015/2016,18,2015-12-06 00:00:00,1992136,7896,9956,Lugano,LUG,Grasshopper Club Zürich,GRA
25978,24558,24558,2015/2016,32,2016-05-08 00:00:00,1992206,7896,9956,Lugano,LUG,Grasshopper Club Zürich,GRA


In [8]:
df = pd.merge(df, matches, left_on='match_api_id', right_on='match_api_id')
df.tail()

,match_api_id,home_team_buildUpPlaySpeed,home_team_buildUpPlayPassing,home_team_chanceCreationPassing,home_team_chanceCreationCrossing,home_team_chanceCreationShooting,home_team_defencePressure,home_team_defenceAggression,home_team_defenceTeamWidth,home_team_avg_shots,...,league_id,season,stage,date,home_team_api_id,away_team_api_id,home_team_long_name,home_team_short_name,away_team_long_name,away_team_short_name
13209,2030167.0,57.0,50.0,61.0,62.0,49.0,49.0,42.0,52.0,11.158163,...,21518,2015/2016,9,2015-10-25 00:00:00,9906,10267,Atlético Madrid,AMA,Valencia CF,VAL
13210,2030168.0,42.0,36.0,41.0,40.0,50.0,47.0,37.0,52.0,9.678322,...,21518,2015/2016,9,2015-10-24 00:00:00,9864,9783,Málaga CF,MAL,RC Deportivo de La Coruña,COR
13211,2030169.0,46.0,39.0,50.0,60.0,36.0,49.0,57.0,50.0,10.117647,...,21518,2015/2016,9,2015-10-26 00:00:00,8315,9869,Athletic Club de Bilbao,BIL,Real Sporting de Gijón,SPG
13212,2030170.0,57.0,59.0,61.0,63.0,51.0,48.0,45.0,62.0,9.684211,...,21518,2015/2016,9,2015-10-24 00:00:00,7878,8603,Granada CF,GRA,Real Betis Balompié,BET
13213,2030171.0,29.0,44.0,44.0,44.0,47.0,54.0,52.0,63.0,11.293478,...,21518,2015/2016,9,2015-10-23 00:00:00,8370,8558,Rayo Vallecano,RAY,RCD Espanyol,ESP


In [9]:
df.to_csv("./dataset/temp_data.csv", index = None)